In [4]:
# 10 Gene Network, 10 indices unknown

import numpy
import numpy as np
from itertools import product
import pickle
from scipy.stats import norm
import pickle
import random

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, RBF

from AFs import knowledge_gradient, expected_improvement, probability_of_improvement, maxvalue_entropy_search, upper_confidence_bound

from time import time

# # Generating Connectivity Matrix
#
# combs = product(range(3), repeat=10)
# Combinations = numpy.array(list(combs))-1
#
# ConnectivitySpace = numpy.zeros((59049, 10*10))
#
OptimalC = np.array(np.mat('+1 -1 -1 0 0 0 0 0 0 0;'
                           '0 0 0 -1 -1 -1 0 0 0 0;'
                           '0 +1 +1 +1 +1 0 0 +1 0 0;'
                           '0 0 0 0 +1 +1 0 0 0 0;'
                           '0 -1 -1 0 -1 0 0 0 0 0;'
                           '0 -1 -1 -1 -1 +1 0 -1 +1 0;'
                           '0 0 0 0 0 -1 0 +1 +1 -1;'
                           '0 0 0 0 0 -1 -1 0 -1 -1;'
                           '0 0 0 0 0 -1 0 0 +1 0;'
                           '0 -1 -1 -1 0 0 +1 -1 +1 0').T) # true #Transpose!!!!

OptimalCVector = numpy.reshape(OptimalC,(1,10*10))
#
# # for i in range(59049):
# #
# #         temp = OptimalCVector
# #
# #         temp[0, (2 - 1) * 10 + 1 - 1] = Combinations[i, 0]
# #         temp[0, (4 - 1) * 10 + 2 - 1] = Combinations[i, 1]
# #         temp[0, (8 - 1) * 10 + 3 - 1] = Combinations[i, 2]
# #         temp[0, (5 - 1) * 10 + 4 - 1] = Combinations[i, 3]
# #         temp[0, (3 - 1) * 10 + 5 - 1] = Combinations[i, 4]
# #         temp[0, (9 - 1) * 10 + 6 - 1] = Combinations[i, 5]
# #         temp[0, (6 - 1) * 10 + 7 - 1] = Combinations[i, 6]
# #         temp[0, (9 - 1) * 10 + 8 - 1] = Combinations[i, 7]
# #         temp[0, (6 - 1) * 10 + 9 - 1] = Combinations[i, 8]
# #         temp[0, (3 - 1) * 10 + 10 - 1] = Combinations[i, 9]
# #
# #         ConnectivitySpace[i,:] = temp
#
#
# ConnectivitySpace[0,:] = OptimalCVector

# #Generating x
# dim = 100
# number_of_states = 10
# x = np.array(np.zeros((number_of_states, dim)).T)
# # initial x0
# x[0, :] = np.array([0, 0, 1, 0, 1, 1, 0, 0, 1, 1])


# for i in range(dim-1):
#     xtemp = np.atleast_2d(x[i, :])
#     b = np.random.binomial(1, 0.3, size=(1, number_of_states))  # it works but numpy website says update with new code
#     x[i+1, :] = (np.matmul(xtemp, (OptimalC.T)) > 0)
#     x[i+1, :] = 1 * np.logical_xor(x[i+1, :], b) # *1 is for converting bool to int

# # Saving the objects:
# with open('OBSGRNVals10GeneB31100Samples.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
#     pickle.dump(x, f)

# #Saving the objects:
# with open('BOValsNewGRN2.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
#     pickle.dump(ConnectivitySpace, f)

start = time()

with open('BOValsNewGRN2.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    ConnectivitySpace = pickle.load(f)

with open('OBSGRNVals10GeneB12100Samples2.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    x = pickle.load(f)
    
with open('50MissingData1.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    MissingData = pickle.load(f)

# Generate A

combs = product(range(2), repeat=10)

combs = list(combs)

A = numpy.array(combs)

Pvalue = 0.01
dna_dsb = np.array(np.mat('0 0 0 0 0 0 0 0 0 0'))
myscale = 50
xycount = 100

# Generate M

posterior = (1 / 1024) * np.ones((1, 1024))

M = np.array(np.zeros((1024, 1024)).T)

combs = product(range(2), repeat=10)

numcombs = numpy.array(list(combs))

xnew2 = np.array(np.zeros((10, 1)).T)

# test a random C

randomCIndex = random.choice(range(59049))
randomCVector = np.atleast_2d(ConnectivitySpace[randomCIndex,:])
randomC = np.reshape(ConnectivitySpace[randomCIndex,:], (10,10))

for i in range(1024):
    xtemp = np.atleast_2d(numcombs[i, :])
    xnew2 = 1 * (np.matmul(xtemp, (randomC).T + dna_dsb) > 0)
    # print (xnew2)
    for t in range(1024):

        xtemp2 = np.atleast_2d(numcombs[t, :])

        M[t, i] = 1

        for k in range(10):

            if xtemp2[0, k] == xnew2[0, k]:
                temp1 = 1 - Pvalue
            else:
                temp1 = Pvalue

            M[t, i] = M[t, i] * temp1

missflag = 0
LogMarginal = 0

for i in range(xycount):

    if i in MissingData:

        pred_posterior = np.matmul(posterior,M.T)
        #pred_posterior = pred_posterior / np.sum(pred_posterior)
        #print(np.sum(pred_posterior))
        # newstate = (newstate) / (np.sum(newstate))
        posterior = pred_posterior
        #print(np.sum(posterior))

    else:

        itemindex = np.array((A == x[i, :]).all(axis=1).nonzero())

        #LogMarginalTrain = LogMarginalTrain + np.log10(pred_posterior[0, itemindex])
        pred_posterior = np.matmul(posterior, M.T)
        #pred_posterior = pred_posterior/np.sum(pred_posterior)
        #print(np.sum(pred_posterior))
        # newstate = (newstate) / (np.sum(newstate))
        LogMarginal = LogMarginal + np.log10(pred_posterior[0, itemindex])
        # missflag = 0

        posterior = numpy.zeros((1,1024))
        posterior[0, itemindex] = 1

LogMarginalTrain = np.atleast_2d(LogMarginal)


kernel = ConstantKernel(0.5 ** 2, (0.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001, 30.01)) * RBF(12, (0.000001, 40))  # ConstantKernel(400 ** 2, (0.01, 1000)) * RBF(60, (0.01, 100))  #Changed # considering 20 Samples!, Think more about these!!
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=50, normalize_y=True)

gp.fit(randomCVector, LogMarginalTrain)

# updated_mean, updated_sigma = gp.predict(ConnectivitySpace, return_std=True) # comment for now

CTrain = randomCVector

num_iter = 100

for i in range(num_iter):

    print(i)
    # Predict Y_mean and Y_cov
    # y_mean, y_std = gp.predict(ConnectivitySpace, return_std=True)

    acquisition_values = expected_improvement(ConnectivitySpace, CTrain, gp)

    argmaxes = np.argwhere(acquisition_values == np.amax(acquisition_values))
    randargmax = np.atleast_2d(random.choice(argmaxes))
    Cnext = np.atleast_2d(ConnectivitySpace[randargmax[0,0],:]) #.T

    CTrain = np.vstack((CTrain, Cnext)) #vstack

    
    # Run the filter and Add the LML to the LogMarginalTrain Vector

    CnextReshape = numpy.reshape(Cnext, (10 , 10))
    
    # Generate M

    posterior = (1 / 1024) * np.ones((1, 1024))

    M = np.array(np.zeros((1024, 1024)).T)

    combs = product(range(2), repeat=10)

    numcombs = numpy.array(list(combs))

    xnew2 = np.array(np.zeros((10, 1)).T)

    for i in range(1024):
        xtemp = np.atleast_2d(numcombs[i, :])
        xnew2 = 1 * (np.matmul(xtemp, (CnextReshape).T + dna_dsb) > 0)
        # print (xnew2)
        for t in range(1024):

            xtemp2 = np.atleast_2d(numcombs[t, :])

            M[t, i] = 1

            for k in range(10):

                if xtemp2[0, k] == xnew2[0, k]:
                    temp1 = 1 - Pvalue
                else:
                    temp1 = Pvalue

                M[t, i] = M[t, i] * temp1

    missflag = 0
    LogMarginal = 0

    for i in range(xycount):

        if i in MissingData:

            pred_posterior = np.matmul(posterior,M.T)
            #pred_posterior = pred_posterior / np.sum(pred_posterior)
            #print(np.sum(pred_posterior))
            # newstate = (newstate) / (np.sum(newstate))
            posterior = pred_posterior
            #print(np.sum(posterior))

        else:

            itemindex = np.array((A == x[i, :]).all(axis=1).nonzero())

            #LogMarginalTrain = LogMarginalTrain + np.log10(pred_posterior[0, itemindex])
            pred_posterior = np.matmul(posterior, M.T)
            #pred_posterior = pred_posterior/np.sum(pred_posterior)
            #print(np.sum(pred_posterior))
            # newstate = (newstate) / (np.sum(newstate))
            LogMarginal = LogMarginal + np.log10(pred_posterior[0, itemindex])
            # missflag = 0

            posterior = numpy.zeros((1,1024))
            posterior[0, itemindex] = 1

    LogMarginalTrain = np.vstack((LogMarginalTrain, np.atleast_2d(LogMarginal)))
    gp.fit(CTrain, LogMarginalTrain)

    # updated_mean, updated_sigma = gp.predict(ConnectivitySpace, return_std=True) # comment for now
#Saving the objects:
with open('10Gene10UnknownB12100Samples50MissingOnlyBO2.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump([LogMarginalTrain, CTrain], f)

end = time()
print(f'It took {end - start} seconds!')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65


/home/alali.m/.conda/envs/BOGRNTesting/lib/python3.8/site-packages/sklearn/gaussian_process/_gpr.py:362: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


66
67
68


/home/alali.m/.conda/envs/BOGRNTesting/lib/python3.8/site-packages/sklearn/gaussian_process/_gpr.py:362: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


69


...

94
95
96
97


/home/alali.m/.conda/envs/BOGRNTesting/lib/python3.8/site-packages/sklearn/gaussian_process/_gpr.py:362: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


98


/home/alali.m/.conda/envs/BOGRNTesting/lib/python3.8/site-packages/sklearn/gaussian_process/_gpr.py:362: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


99
It took 1367.6683676242828 seconds!
